In [1]:
# Import libraries
from flair.data import Sentence
from flair.models import SequenceTagger
from cassis import *

/home/allan/snap/jupyter/common/lib/python3.7/site-packages/joblib/_multiprocessing_helpers.py:45: UserWarning: [Errno 13] Permission denied.  joblib will operate in serial mode
  warnings.warn('%s.  joblib will operate in serial mode' % (e,))


In [2]:
pip freeze > requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [3]:
tsfile = 'ts1.xml'
csfile = './output1.xmi/document.txt.xmi'

TOKEN_TYPE = "de.tudarmstadt.ukp.dkpro.core.api.segmentation.type.Token"
SENTENCE_TYPE = "de.tudarmstadt.ukp.dkpro.core.api.segmentation.type.Sentence"

# Load Typesystem
with open(tsfile, 'rb') as f:
    typesystem = load_typesystem(f)

# Load Cas
with open(csfile, 'rb') as f:
    cas = load_cas_from_xmi(f, typesystem=typesystem)

In [5]:
tokens = list(cas.select(TOKEN_TYPE))
words = [cas.get_covered_text(token) for token in tokens]
# tokens

/snap/jupyter/6/lib/python3.7/site-packages/ipykernel_launcher.py:2: DeprecatedWarning: get_covered_text is deprecated. Use annotation.get_covered_text()
  


In [6]:
sentence = ''
for word in words:
    sentence += word + ' '

sentence = Sentence(sentence)
model_name = 'ner'
nlp = SequenceTagger.load(model_name)

nlp.predict(sentence)

2020-06-16 16:25:20,709 loading file /home/allan/snap/jupyter/6/.flair/models/en-ner-conll03-v0.4.pt


[Sentence: "Courtney de Souza has been living with her daughter Susan in Berlin for about 6 years ."   [− Tokens: 17  − Token-Labels: "Courtney <B-PER> de <I-PER> Souza <E-PER> has been living with her daughter Susan <S-PER> in Berlin <S-LOC> for about 6 years ."]]

In [7]:
# Get annotation type
AnnotationType = typesystem.get_type(TOKEN_TYPE)
NERType = typesystem.get_type("de.tudarmstadt.ukp.dkpro.core.api.ner.type.NamedEntity")

In [8]:
i = 0
for span in sentence.get_spans('ner'):
    cas_sentence = AnnotationType(begin = span.start_pos, 
                                  end = span.end_pos)
    if sentence.get_spans('ner')[i].tag == 'PER':
        val = 'person'
    if sentence.get_spans('ner')[i].tag == 'LOC':
        val = 'location'
    ner_annotation = NERType(begin = span.start_pos,
                             end = span.end_pos,
                             value = val)
    cas.add_annotation(cas_sentence)
    cas.add_annotation(ner_annotation)
    i=i+1
xmi = cas.to_xmi('./output2.xmi', pretty_print=True)